In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import pandas as pd
import xarray as xr
import rioxarray

import codebase

## Check metadata of DEM. Want to find units and reference value

In [5]:
res_shp = codebase.load_data.load_GRanD()

In [6]:
dam_name = 'Hoover'
subset_gpd = codebase.area_subsets.check_for_multiple_dams(dam_name,res_shp)
subset_bbox = subset_gpd.geometry.buffer(0).bounds
dem_DA = codebase.load_data.load_DEM_subset_as_rxrDA(subset_bbox)

In [7]:
dem_DA

<xarray.DataArray (lat: 55, lon: 98)>
array([[ 987.53045412,  969.79296571,  946.72921483, ..., 1470.38425046,
        1465.00233566, 1469.4108313 ],
       [ 907.45159379,  920.56432272,  988.33743756, ..., 1440.64440223,
        1447.92755293, 1451.14704499],
       [ 858.63924561,  923.07848935, 1014.14195745, ..., 1404.82680196,
        1415.93721157, 1407.33142665],
       ...,
       [ 641.08900364,  644.37957993,  654.04210321, ...,  924.03486371,
         968.38722582, 1013.88984158],
       [ 660.72744196,  682.84490731,  682.97765448, ..., 1018.38934307,
         944.81193003, 1014.4296659 ],
       [ 694.18572515,  784.30122756,  715.92518227, ..., 1082.08878469,
         999.17180333,  967.66697302]])
Coordinates:
  * lat          (lat) float64 36.0 36.01 36.02 36.03 ... 36.52 36.53 36.54
  * lon          (lon) float64 -114.9 -114.9 -114.9 ... -113.9 -113.9 -113.9
    spatial_ref  int64 0

## Checking that load functions work
The full load and subset load both work.

In [2]:
dem_test , lat_test , lon_test = codebase.load_data.load_DEM_full_as_nparray()
dem_DA = codebase.load_data.load_DEM_full_as_rxrDA()

## Load local subset

In [6]:
import xarray as xr
from codebase import volume_pipeline

In [7]:
indira = xr.open_dataarray('../indirasagar_dem_subset.nc')

In [8]:
volume_pipeline.project_DA_from_crs_code(indira,54017)

ImportError: DLL load failed while importing _version: The specified procedure could not be found.